In [36]:
import re
import json
import pymorphy3
import tokenize_uk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [37]:
df = pd.read_csv('../../data/prepared_candidates/candidates_prepared.csv')
df.head()

,Position,Moreinfo,Looking For,Highlights,Primary Keyword,English Level,Experience Years,CV,CV_lang,id
0,1c,"Розробник 1с/BAS, всі українські конфігурації,...",NaN,NaN,Other,intermediate,11.0,"\nРозробник 1с/BAS, всі українські конфігураці...",uk,dcaeffc4-1d1f-5754-a567-f478f0c8aa6f
1,1c developer,Еко - маркет Торгівля роздрібна 1 рік 9 місяці...,NaN,Hillel IT school Java Basic and Java Pro 2022\...,SQL,pre,11.0,Hillel IT school Java Basic and Java Pro 2022\...,uk,2767ca2d-4bca-5b41-b9e0-e42ad5700b6f
2,1C developer (purpose: switch to Golang/Ruby),Займаюся автоматизацією підприємств вже понад ...,"Найголовніше - це атмосфера в колективі, також...","Самостійно приймати рішення, брати відповідаль...",Ruby,upper,7.0,"Самостійно приймати рішення, брати відповідаль...",uk,ff07df1f-5268-5865-8f30-1cc660786494
3,1C програміст,Шукаю роботу з метою розвитку та вдосконалення...,В першу чергу мене цікавить можливість розвитк...,"Пройшов курси: «1С за 21 день, 1С 8.3» та «1С ...",SQL,intermediate,0.5,"Пройшов курси: «1С за 21 день, 1С 8.3» та «1С ...",uk,da4b79b1-fbc5-5dfe-bba0-58339f262d11
4,1C програміст,Шукаю роботу по напрямку junior С++. Маю досві...,"Цікаві проекти, саморозвиток.",NaN,Data Engineer,intermediate,6.0,\nШукаю роботу по напрямку junior С++. Маю дос...,uk,b118a952-cbe6-58ac-824a-10de5322570c


In [38]:
# check phone number
def detect_phone(text):
    if re.search(r'380\d{9}', text.replace(' ','')) or re.search(r'0\d{9}', text.replace(' ','')):
        return 1
    return 0

def get_phone(text):
    if re.search(r'380\d{9}', text.replace(' ','')):
        return re.search(r'380\d{9}', text.replace(' ','')).group()
    elif re.search(r'0\d{9}', text.replace(' ','')):
        return re.search(r'0\d{9}', text.replace(' ','')).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_phone(cv_text)
print("Count of CVs with phone number in CVs:", count)
print("Percentage of CVs with phone number in CVs:", 100*count/len(df))

# get phone
df['phone'] = df['CV'].apply(get_phone)
# show 10 random phone
df['phone'].unique()[np.random.randint(0, len(df['phone'].unique()), 10)]

100%|██████████| 234896/234896 [00:02<00:00, 87121.07it/s]


Count of CVs with phone number in CVs: 97
Percentage of CVs with phone number in CVs: 0.04129487092159934


array(['380679171057', '0180129769', '380662442517', '0678528390',
       '0615358656', '380958966650', '380633150119', '0662569843',
       '380966772305', '0713095971'], dtype=object)

In [39]:
# check email
def detect_email(text):
    if re.search(r'.+[@ _]gmail\.com', text.replace(' ','')) or re.search(r'.+[@ _]ukr\.net', text.replace(' ','')) or re.search(r'.+[@ _]mail\.ru', text.replace(' ','')):
        return 1
    return 0

def get_email(text):
    if re.search(r'.+[@ _]gmail\.com', text.replace(' ','')):
        return re.search(r'.+[@ _]gmail\.com', text.replace(' ','')).group()
    elif re.search(r'.+[@ _]ukr\.net', text.replace(' ','')):
        return re.search(r'.+[@ _]ukr\.net', text.replace(' ','')).group()
    elif re.search(r'.+[@ _]mail\.ru', text.replace(' ','')):
        return re.search(r'.+[@ _]mail\.ru', text.replace(' ','')).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_email(cv_text)
print("Count of CVs with email in CVs:", count)
print("Percentage of CVs with email in CVs:", 100*count/len(df))

# get email
df['email'] = df['CV'].apply(get_email)
# show 10 random emails
df['email'].unique()[np.random.randint(0, len(df['email'].unique()), 10)]

100%|██████████| 234896/234896 [09:16<00:00, 421.83it/s]


Count of CVs with email in CVs: 99
Percentage of CVs with email in CVs: 0.042146311559158094


array(['Librarymanagement(loginandpasswordfromadminpartmr.mastercredit_gmail.com',
       'YevheniyaSafonyuk|+393923384003(Telegram,WhatsApp)|eugeniasaf_gmail.com',
       'Email:romleon36_gmail.com',
       'Feelfreetoreachouttomeatroksolana.tylipska_gmail.com',
       'bodya.yarovenko_gmail.com',
       'IhavebeendevelopingITbusinessformorethan15years.Gainedexperienceinimplementinglarge-scaleprojectsinalargecompanyfortheproductionandsupplyofITequipmentandsoftware,asacorporatesalesmanager,thenasheadofcorporatesales.TodaymyworkisrelatedtothedevelopmentofsoftwaresalesintheCIScountriesintheB2Bformat.Iamveryinspiredbytheprocessofbuildinglong-termrelationshipswithclientsandpartners,andconcludingagreements.Andthemoredifficultthegoal,themorecreativityandenergyintheprocessofachievingit.Iamnotlimitedtoonearea,Iaminterestedinlarge-scaleinnovativeprojectsthatcreatevalueforbusinessandpeople.Opentocooperationandnewinterestingtasks,readytoimplementambitiousprojectsasabusinessdevelopmentmanagerandsa

In [40]:
# check вул or street
def detect_street(text):
    if re.search(r'вул\.', text) or re.search(r'home address', text):
        return 1
    return 0

def get_street(text):
    if re.search(r'вул\.', text):
        return re.search(r'вул\..+', text).group()
    elif re.search(r'home address', text):
        return re.search(r'home address.+', text).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_street(cv_text)
print("Count of CVs with вул. in CVs:", count)
print("Percentage of CVs with вул. in CVs:", 100*count/len(df))

# get street
df['street'] = df['CV'].apply(get_street)
# show 10 random streets
df['street'].unique()[:10]

100%|██████████| 234896/234896 [00:00<00:00, 438323.80it/s]


Count of CVs with вул. in CVs: 3
Percentage of CVs with вул. in CVs: 0.0012771609563381242


array([None,
       'вул. Соборна 43,  до роботи залучав інших осіб у якості працівників), продовжуючи працювати в сфері реклами (арбітраж трафіку).\r',
       'вул. В.Винниченка, 30, Львів\r', 'вул. Варненська, 2а).\r'],
      dtype=object)

In [41]:
# check ІПН, РНОКПП, УНЗР, SNN
def detect_ipn(text):
    if re.search(r'ІПН', text) or re.search(r'РНОКПП', text) or re.search(r'УНЗР', text) or re.search(r'SNN:', text):
        print(text)
        return 1
    return 0

def get_ipn(text):
    if re.search(r'ІПН', text):
        return re.search(r'ІПН.+', text).group()
    elif re.search(r'РНОКПП', text):
        return re.search(r'РНОКПП.+', text).group()
    elif re.search(r'УНЗР', text):
        return re.search(r'УНЗР.+', text).group()
    elif re.search(r'SNN:', text):
        return re.search(r'SNN:.+', text).group()
    return None

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_ipn(cv_text)
print("Count of CVs with ІПН in CVs:", count)
print("Percentage of CVs with ІПН in CVs:", 100*count/len(df))

# get ІПН
df['ipn'] = df['CV'].apply(get_ipn)
# show 10 random ІПН
df['ipn'].unique()[:10] # noy find

 26%|██▋       | 61978/234896 [00:00<00:00, 318121.93it/s]

Розробив в Excel форму довідки про доходи, яка автоматично заповняється по ІПН.
Верстка газети "Тернопіль вечірній" QuarkXPress, PageMaker, PhotoShop, CorelDraw. (2 роки)
Бухгалтер (4 роки)
Командний проект сайт готелю (HTML, CSS, SASS):
верстка сторінок, меню ресторану, футер.
Командний проект Filmoteka (HTML, CSS, javaScript)



100%|██████████| 234896/234896 [00:00<00:00, 349916.25it/s]


Count of CVs with ІПН in CVs: 1
Percentage of CVs with ІПН in CVs: 0.0004257203187793747


array([None, 'ІПН.'], dtype=object)

In [42]:
# check social media links
def detect_social(text):
    if re.search(r"w{3}\.linkedin", text) or re.search(r"w{3}\.facebook", text) or re.search(r"w{3}\.instagram", text) or re.search(r"w{3}\.twitter", text) or re.search(r"w{3}\.vk", text) or re.search(r"w{3}\.telegram", text) or re.search(r"w{3}\.viber", text) or re.search(r"w{3}\.whatsapp", text):
        return 1
    return 0

def get_social(text):
    social = []
    if re.search(r"w{3}\.linkedin", text):
        social.append(re.search(r"w{3}\.linkedin.+", text).group())
    if re.search(r"w{3}\.facebook", text):
        social.append(re.search(r"w{3}\.facebook.+", text).group())
    if re.search(r"w{3}\.instagram", text):
        social.append(re.search(r"w{3}\.instagram.+", text).group())
    if re.search(r"w{3}\.twitter", text):
        social.append(re.search(r"w{3}\.twitter.+", text).group())
    if re.search(r"w{3}\.vk", text):
        social.append(re.search(r"w{3}\.vk.+", text).group())
    if re.search(r"w{3}\.telegram", text):
        social.append(re.search(r"w{3}\.telegram.+", text).group())
    if re.search(r"w{3}\.viber", text):
        social.append(re.search(r"w{3}\.viber.+", text).group())
    if re.search(r"w{3}\.whatsapp", text):
        social.append(re.search(r"w{3}\.whatsapp.+", text).group())
    return str(social)

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=detect_social(cv_text)
print("Count of CVs with social media links in CVs:", count)
print("Percentage of CVs with social media links in CVs:", 100*count/len(df))

# get social media links
df['social'] = df['CV'].apply(get_social)
# show 10 random social media links
df['social'].unique()[np.random.randint(0, len(df['social'].unique()), 10)]

  0%|          | 0/234896 [00:00<?, ?it/s]

100%|██████████| 234896/234896 [00:17<00:00, 13799.46it/s]


Count of CVs with social media links in CVs: 72
Percentage of CVs with social media links in CVs: 0.03065186295211498


array(["['www.linkedin.com/in/denis-gabaydulin-4bb1996a?trk=nav_responsive_tab_profile']",
       "['www.linkedin.com/in/tanya-saikevych/']",
       "['www.linkedin.com/in/aaadezign-drozd-242180b4']",
       "['www.linkedin.com/in/antonymasoka/']",
       "['www.linkedin.com/in/vyacheslavshcherbak/']",
       "['www.facebook.com/myprovincel/']",
       "['www.linkedin.com/in/igorzarytovskykh/']",
       "['www.linkedin.com/in/alexshvets/']",
       "['www.facebook.com/anka.lyah\\r', 'www.instagram.com/hannaliakh/']",
       "['www.linkedin.com/in/yuriy-kurylo-963067a4?trk=nav_responsive_tab_profile']"],
      dtype=object)

In [43]:
# find socila media nicknames
def find_social_nickname(text):
    text = text.lower()
    if re.search(r"linkedin:", text) or re.search(r"facebook:", text) or re.search(r"instagram:", text) or re.search(r"twitter:", text) or \
        re.search(r"vk:", text) or re.search(r"vkontakte:", text) or re.search(r"telegram:", text) or re.search(r"viber:", text) or re.search(r"whatsapp:", text):
        return 1
    return 0

def get_social_nickname(text):
    social = []
    if re.search(r"linkedin:.+", text):
        social.append(re.search(r"linkedin:.+", text).group())
    if re.search(r"facebook:.+", text):
        social.append(re.search(r"facebook:.+", text).group())
    if re.search(r"instagram:.+", text):
        social.append(re.search(r"instagram:.+", text).group())
    if re.search(r"twitter:.+", text):
        social.append(re.search(r"twitter:.+", text).group())
    if re.search(r"vk:.+", text):
        social.append(re.search(r"vk:.+", text).group())
    if re.search(r"vkontakte:.+", text):
        social.append(re.search(r"vkontakte:.+", text).group())
    if re.search(r"telegram:.+", text):
        social.append(re.search(r"telegram:.+", text).group())
    if re.search(r"viber:.+", text):
        social.append(re.search(r"viber:.+", text).group())
    if re.search(r"whatsapp:.+", text):
        social.append(re.search(r"whatsapp:.+", text).group())
    return str(social)

count = 0
for cv_text in tqdm(df['CV'].values):
    count+=find_social_nickname(cv_text)
print("Count of CVs with social media nicknames in CVs:", count)
print("Percentage of CVs with social media nicknames in CVs:", 100*count/len(df))

# get social media nicknames
df['social_nickname'] = df['CV'].apply(get_social_nickname)
# show 10 random social media nicknames
df['social_nickname'].unique()[np.random.randint(0, len(df['social_nickname'].unique()), 10)]

100%|██████████| 234896/234896 [00:02<00:00, 84317.90it/s]


Count of CVs with social media nicknames in CVs: 187
Percentage of CVs with social media nicknames in CVs: 0.07960969961174306


array(["['linkedin: iakivkramarenko)\\r']",
       "['linkedin: vadym-kuznietsov-35275a95']", '[]',
       "['linkedin: kochkinanna (for some reason, you cannot insert links in this field).\\r']",
       "['linkedin: /mwlite/in/cristian-machuca-dev']", "['instagram:)']",
       "['telegram: Ainur_Om']", "['instagram:\\r']",
       "['telegram: komil_mustaev']",
       "['linkedin: vadym-kuznietsov-35275a95']"], dtype=object)

In [59]:
df.social = df.social.astype(str)
df = df[(df.social_nickname != '[]') | (df.social != '[]') | (df.ipn.notna()) | (df.street.notna()) | (df.email.notna()) |  (df.phone.notna())]
print(df.shape)
df.head()

(269, 16)


,Position,Moreinfo,Looking For,Highlights,Primary Keyword,English Level,Experience Years,CV,CV_lang,id,phone,email,street,ipn,social,social_nickname
195,2D Artist,"Concept Art, 2D traditional and digital painti...","Шукаю: GameDev, Ілюстрації",Портфоліо:\r\nhttps://www.artstation.com/tsaik...,Design,pre,0.0,Портфоліо:\r\nhttps://www.artstation.com/tsaik...,uk,a4a6d91d-35c4-52d0-81dd-84f708ca4cbe,None,None,None,None,['www.linkedin.com/in/tanya-saikevych/'],[]
1124,ASO Specialist/ Application Project Manager,В червні 2021 року закінчила он-лайн курси в A...,NaN,NaN,Other,pre,1.0,\nВ червні 2021 року закінчила он-лайн курси в...,uk,50be5173-59d8-5fef-bc68-1de0965af1b3,0212294725,None,None,None,[],[]
1523,Business Analyst,Я маю 7 років досвіду роботи в різних компанія...,Я очікую отримати практичні навички роботи біз...,"1. Працюючи в юридичній компанії ""Юридичний Це...",Business Analyst,intermediate,0.5,"1. Працюючи в юридичній компанії ""Юридичний Це...",uk,5bb03043-b226-5806-8f62-855677cfb6dc,None,None,None,None,[],['linkedin: vadym-kuznietsov-35275a95']
2057,copywriter,"Копірайтер\r\nз лютого 2018 Люксоптика, копіра...",NaN,Статті \r\n· https://bit.ly/2GfDuBz \r\...,Marketing,intermediate,1.0,Статті \r\n· https://bit.ly/2GfDuBz \r\...,uk,b78a4f0e-01ce-5394-abfb-0aac9f17486c,None,None,None,None,['www.facebook.com/LINZMASTER/\r'],[]
3022,Digital Marketing / CRM / Junior Frontend & Ja...,"Привіт, Я є Junior Frontend Developer з досвід...",NaN,- Запустив і успішно провів більше 10 навчальн...,Marketing,pre,3.0,- Запустив і успішно провів більше 10 навчальн...,uk,b61fcdee-644b-518f-ba23-c4806afe2ab6,None,Email:alexlanin95_gmail.com,None,None,[],[]


In [61]:
df.to_csv('../data/PII_CV.csv', index=False)

## Delete PII from datasets

In [70]:
data = pd.read_csv('../../data/djinni/candidates.csv')

In [71]:
data

,Position,Moreinfo,Looking For,Highlights,Primary Keyword,English Level,Experience Years
0,=,Маю досвід роботи рекрутером більше 1 року. \r...,NaN,"Пройдений курс ""Introduction to Machine Learni...",Marketing,upper,0.0
1,_,_____,_,_,Other,upper,11.0
2,_,Studied EPAM 'IT switch course' 2022 which inc...,NaN,NaN,Java,intermediate,0.0
3,_,Наразі досвід є лише в якихось університетськи...,NaN,"Навчаюся в університеті на ""відмінно"", легко д...",Sales,pre,0.0
4,__,__,NaN,NaN,NaN,intermediate,6.0
...,...,...,...,...,...,...,...
295089,Юрист з інтелектуальної власності,I am an IP Lawyer with a three years’ experien...,NaN,I have experience in various intellectual prop...,Lead,upper,3.0
295090,"Юрист, Помощник Юриста","Бухгалтер з первинної документації, міжнародна...",В мене є досвід співпраці з адвокатською орган...,Диплом магістра з правознавства з відзнакою;\r...,Other,intermediate,0.0
295091,Юрист/Старший юрист/Керівник відділу,I am international lawyer with 9+ years of exp...,NaN,"Two years ago, I decided to upgrade myself in ...",Other,fluent,9.0
295092,"Юрист, юрисконсульт",Проводила державну реєсуацію юридичних фактів ...,NaN,NaN,Other,no_english,6.0


In [72]:
df = df[data.columns]

In [74]:
#delete all rows from data which we have in df. for deleting use match by all columns
data = data[~data.apply(tuple,1).isin(df.apply(tuple,1))]


In [75]:
data

,Position,Moreinfo,Looking For,Highlights,Primary Keyword,English Level,Experience Years
0,=,Маю досвід роботи рекрутером більше 1 року. \r...,NaN,"Пройдений курс ""Introduction to Machine Learni...",Marketing,upper,0.0
1,_,_____,_,_,Other,upper,11.0
2,_,Studied EPAM 'IT switch course' 2022 which inc...,NaN,NaN,Java,intermediate,0.0
3,_,Наразі досвід є лише в якихось університетськи...,NaN,"Навчаюся в університеті на ""відмінно"", легко д...",Sales,pre,0.0
4,__,__,NaN,NaN,NaN,intermediate,6.0
...,...,...,...,...,...,...,...
295089,Юрист з інтелектуальної власності,I am an IP Lawyer with a three years’ experien...,NaN,I have experience in various intellectual prop...,Lead,upper,3.0
295090,"Юрист, Помощник Юриста","Бухгалтер з первинної документації, міжнародна...",В мене є досвід співпраці з адвокатською орган...,Диплом магістра з правознавства з відзнакою;\r...,Other,intermediate,0.0
295091,Юрист/Старший юрист/Керівник відділу,I am international lawyer with 9+ years of exp...,NaN,"Two years ago, I decided to upgrade myself in ...",Other,fluent,9.0
295092,"Юрист, юрисконсульт",Проводила державну реєсуацію юридичних фактів ...,NaN,NaN,Other,no_english,6.0


In [76]:
data.to_csv('../../data/djinni/candidates.csv', index=False)

In [81]:
# delete from preprocessed datasets
data = pd.read_csv('../../data/prepared_candidates/candidates_prepared.csv')
print(data.shape)
df = pd.read_csv('../data/PII_CV.csv')
# drop by id
data = data[~data.id.isin(df.id)]
print(data.shape)
data.to_csv('../../data/prepared_candidates/candidates_prepared.csv', index=False)

(234896, 10)
(234627, 10)


In [82]:
# delete from preprocessed datasets
data = pd.read_csv('../../data/prepared_candidates/intermediate_candidates_prepared.csv')
print(data.shape)
df = pd.read_csv('../data/PII_CV.csv')
# drop by id
data = data[~data.id.isin(df.id)]
print(data.shape)
data.to_csv('../../data/prepared_candidates/intermediate_candidates_prepared.csv', index=False)

(241977, 10)
(241708, 10)
